In [1]:
import json

dev = []
with open("data/TreePos_BI/dev.json","r") as f:
    for line in f:
        dev.append(json.loads(line))

for d in dev:
    d["tree_pos"] = [d["tags"][i].split("_")[0] for i in range(len(d["tags"]))]
    d["ner_pos"] = [d["tags"][i].split("_")[1] for i in range(len(d["tags"]))]

In [2]:
import json

train = []
with open("data/TreePos_BI/train.json","r") as f:
    for line in f:
        train.append(json.loads(line))

for d in train:
    d["tree_pos"] = [d["tags"][i].split("_")[0] for i in range(len(d["tags"]))]
    d["ner_pos"] = [d["tags"][i].split("_")[1] for i in range(len(d["tags"]))]

In [36]:
# d= {'tokens': ['The',...,'.'],'tags': ['DT_O',...,'._O'],'tree_pos': ['DT',....'.'], 'ner_pos': ['O',...'O']}
def make_data2list_Simple(data):
    data_list = []
    for d in dev:            
        new_sent = []
        before_tag = ""
        new_token = []
        for i in range(len(d["tags"])):

            if d["ner_pos"][i] == "O" or i == len(d["tokens"])-1:#固有表現タグがO　または　最後のトークンの時
                if d["ner_pos"][i] == before_tag:#1
                    new_token.append(d["tokens"][i])
                    new_sent.append([new_token,d["ner_pos"][i]])
                    new_token = []
                else:
                    new_token = []#2
                    new_token.append(d["tokens"][i])
                    new_sent.append([new_token,d["ner_pos"][i]])
                    new_token = []
                    
            else:#固有表現タグがBI#3
                if d["ner_pos"][i] == before_tag:
                    new_token.append(d["tokens"][i])
                    new_token = []
                else:#4
                    new_token.append(d["tokens"][i])
                    new_sent.append([new_token,d["ner_pos"][i]])
            before_tag = d["ner_pos"][i]
        data_list.append(new_sent)
    return data_list
# dev_list= [['Danish', 'rigsdaler'], 'other-currency'],[['='], 'O'],...


def make_data2list_TreePos(data):
    data_list = []
    for d in dev:            
        new_sent = []
        before_tag = ""
        new_token = []
        new_tag = []
        for i in range(len(d["tags"])):

            if d["ner_pos"][i] == "O" or i == len(d["tokens"])-1:#固有表現タグがO　または　最後のトークンの時
                if d["ner_pos"][i] == before_tag:#1
                    new_token.append(d["tokens"][i])
                    new_tag.append(d["tags"][i])
                    
                    new_tag = list(set(new_tag))  #new_tagの重複削除
                    new_tag.sort()                #new_tagの整列
                    new_sent.append([new_token,"__".join(new_tag)])
                    
                    new_token = []
                    new_tag = []
                else:
                    new_token = []
                    new_tag = []
                    
                    new_token.append(d["tokens"][i])
                    new_tag.append(d["tags"][i])
                    
                    new_tag = list(set(new_tag))  #new_tagの重複削除
                    new_tag.sort()                #new_tagの整列
                    new_sent.append([new_token,"__".join(new_tag)])
                    
                    new_token = []
                    new_tag = []
                    
            else:#固有表現タグがBI
                if d["ner_pos"][i] == before_tag:
                    new_token.append(d["tokens"][i])
                    new_tag.append(d["tags"][i])
                    
                    new_token = []
                    new_tag = []
                else:
                    new_token.append(d["tokens"][i])
                    new_tag.append(d["tags"][i])
                    
                    new_tag = list(set(new_tag))  #new_tagの重複削除
                    new_tag.sort()                #new_tagの整列
                    new_sent.append([new_token,"__".join(new_tag)])#new_tagを__で結合
                    
            before_tag = d["ner_pos"][i]
        data_list.append(new_sent)
    return data_list

#交換先辞書の作成
def make_alterna_labels(data_list):
    label_list = set()
    for s in data_list:
        for t in s:
            if t[1] != "O" or "_O" not in t[1][0]:
                label_list.add(t[1])
    
    label_list = list(label_list)
    alterna_labels = dict()
    for label in label_list:
        alterna_labels[label] = set()
    for s in data_list:
        for t in s:
            token = "_".join(t[0])
            tag = t[1]
            if tag in label_list:
                alterna_labels[tag].add(token)
    
    new_alterna_labels = dict()
    for k,v in alterna_labels.items():#候補が空白or1つのtagは削除
        if v==set() or len(v) == 1:
            pass
        else:
            new_alterna_labels[k] = v

    return new_alterna_labels


#Oタグのみデータを排除した増幅候補
def make_data_has_BI(data_list):
    data_has_BI = [] #16342(データ全体は18823)
    for d in data_list:
        tags_list = list(set([i[1] for i in d]))
        if tags_list != ["O"]:
            data_has_BI.append(d)
    return data_has_BI

In [37]:
data_dict = dev[0:100]
print(1)
data_simple = make_data2list_Simple(data_dict)#tagが固有表現情報
data_treepos = make_data2list_TreePos(data_dict)#tagが品詞＋固有表現情報
print(2)
alterna_labels_simple = make_alterna_labels(data_simple)#固有表現情報のみに着目した交換先辞書
alterna_labels_treepos = make_alterna_labels(data_treepos)#品詞＋固有表現情報の交換先辞書
data_has_BI = make_data_has_BI(data_treepos)

1
2


In [41]:
alterna_labels_treepos["NNP_organization-company__NNP_product-other"]

{'Honda_J_V6', 'IBM_Personal_Computer'}

In [42]:
# for s in data_treepos:
#     for row in s:
#         if "__" in row[1]:
#             print(s)
#             break

In [22]:
# data_dict = dev
# data_treepos = make_data2list_TreePos(data_dict)#tagが品詞＋固有表現情報

In [43]:
# cnt = 0
# for s in data_treepos:
#     for row in s:
#         if len(row[0])>1:
#             print(cnt)
#             print(s)
            
#     cnt += 1

In [45]:
# for i in alterna_labels_treepos:
#     print(i)

In [46]:
import random
import json

def random_choice_token(token_set):
    n = random.randint(0,len(token_set)-1)
    token_list=list(token_set)
    return token_list[n]

def Simple_TreePos_BI(datatype,data_dict,n,stoper):
    #[[token_1,...token_n],tag],...[token_1,...token_n],tag]]状態のデータを作成
    data_simple = make_data2list_Simple(data_dict)#tagが固有表現情報
    data_treepos = make_data2list_TreePos(data_dict)#tagが品詞＋固有表現情報
    
    alterna_labels_simple = make_alterna_labels(data_simple)#固有表現情報のみに着目した交換先辞書
    alterna_labels_treepos = make_alterna_labels(data_treepos)#品詞＋固有表現情報の交換先辞書
    data_has_BI = make_data_has_BI(data_treepos)
    #Oタグのみのデータを排除し，交換前候補として格納（treeposのほうが条件が厳しいのでそれを基準とする）
    
    #加工箇所を同じにするため，SimpleとTreePosを同時に作成する
    data_simple_f_path = "data/Simple_BI_x"+str(n)+"/"+datatype+".json"
    data_treepos_f_path = "data/TreePos_BI_x"+str(n)+"/"+datatype+".json"
    memo_f_path = "log/Simple_and_TreePos_BI_x"+str(n)+"_"+datatype+"_memo.txt"

    with open(data_simple_f_path,"w",encoding="utf-8") as data_simple_f, \
    open(data_treepos_f_path,"w",encoding="utf-8") as data_treepos_f, \
    open(memo_f_path,"w",encoding="utf-8") as memo_f:
        
        for d in data_dict:
            write_d = dict()
            write_d["tokens"] = d["tokens"]
            write_d["tags"] = d["ner_pos"]
            data_simple_f.write(json.dumps(write_d))#Simpleにオリジナルデータを先に書込
            data_simple_f.write("\n")
            data_treepos_f.write(json.dumps(write_d))#TreePosにオリジナルデータを先に書込
            data_treepos_f.write("\n")
        
        add_n = n*len(data_dict)-len(data_dict)#拡張データの数
        cnt = 0
        while cnt < add_n:
            if cnt%10 == 0:
                print('\r%d / %d' %(cnt, add_n), end='')

            x = random.randint(0,len(data_has_BI)-1)
            x_s = data_has_BI[x]

            new_d_simple = dict()
            new_d_simple["tokens"] = []
            new_d_simple["tags"] = []
            
            new_d_treepos = dict()
            new_d_treepos["tokens"] = []
            new_d_treepos["tags"] = []
            old_tokens = []
            
            change = False
            for token,tag_treepos in x_s:
                tag_simple = tag_treepos.split("_")[-1]
                old_tokens.extend(token)
                #固有表現であり，厳格な候補であるtreeposに交換対象が存在するとき，50%の確率で入れ替えを発生
                if (tag_simple != "O") and (tag_treepos in alterna_labels_treepos) and (random.random() >= 0.5):
                    
                    #品詞+固有表現情報に基づく交換候補を生成
                    token_simple = random_choice_token(alterna_labels_simple[tag_simple]-{"_".join(token)})
                    token_simple = token_simple.split("_")
                    #交換したtokenを記録
                    new_d_simple["tokens"].extend(token_simple)
                    #新しいtokenの数だけtagを追加
                    for _ in token_simple:
                        new_d_simple["tags"].append(tag_simple)
                        
                    #固有表現情報に基づく交換候補を生成
                    token_treepos = random_choice_token(alterna_labels_treepos[tag_treepos]-{"_".join(token)})
                    token_treepos = token_treepos.split("_")
                    #交換したtokenを記録
                    new_d_treepos["tokens"].extend(token_treepos)
                    #新しいtokenの数だけtagを追加
                    for _ in token_treepos:#
                        new_d_treepos["tags"].append(tag_simple)
                        
                    change = True#書換えフラグ
                else:
                    new_d_simple["tokens"].extend(token)
                    new_d_treepos["tokens"].extend(token)
                    new_d_treepos["tags"].append(tag_simple)

            if change == True:#書換えが実行されたなら現在の一文をデータとして書出（改変前と同じ学習データの混入防止）
                data_simple_f.write(json.dumps(new_d_simple))
                data_simple_f.write("\n")
                data_treepos_f.write(json.dumps(new_d_treepos))
                data_treepos_f.write("\n")
                
                memo_f.write(str(x)+"\t"+" ".join(old_tokens)+"\n")#memoに変更前と変更後を記入
                memo_f.write(" "*len(str(x))+"\t"+" ".join(new_d_simple["tokens"])+"\n")
                memo_f.write(" "*len(str(x))+"\t"+" ".join(new_d_treepos["tokens"])+"\n")
                memo_f.write("\n")
                cnt += 1
            else:
                pass
        memo_f.write(str(cnt)+"sentences augmented!!!! \n")

In [47]:
Simple_TreePos_BI("dev",dev,2,"str")

18820 / 18823

In [48]:
Simple_TreePos_BI("train",train,2,"str")

131710 / 131719

In [ ]:
dev[14]

In [ ]:
t = "rrr_job"
t = "rrr"
t.split("_")

In [41]:
d = [[['The'], 'O'], [['construction'], 'O'], [['of'], 'O'], [['the'], 'O'], [['airport'], 'O'], [['was'], 'O'], [['done'], 'O'], [['mainly'], 'O'], [['by'], 'O'], [['a'], 'O'], [['few'], 'O'], [['state'], 'O'], [['owned'], 'O'], [['construction'], 'O'], [['companies'], 'O'], [['as'], 'O'], [['well'], 'O'], [['as'], 'O'], [['Ekovest', 'Berhad'], 'organization-company'], [['–'], 'O'], [['helmed'], 'O'], [['by'], 'O'], [['Tan', 'Sri'], 'person-other'], [['.'], 'O']]

for token,tag in d:
    print(token)

['The']
['construction']
['of']
['the']
['airport']
['was']
['done']
['mainly']
['by']
['a']
['few']
['state']
['owned']
['construction']
['companies']
['as']
['well']
['as']
['Ekovest', 'Berhad']
['–']
['helmed']
['by']
['Tan', 'Sri']
['.']


In [95]:
for _ in range(20):
    if random.random() >= 0.5:#50%の確率で入れ替えを発生
        print(1)
    else:
        print(-1)

-1
-1
1
1
1
-1
1
-1
1
1
-1
-1
1
-1
-1
-1
-1
1
1
1
